## Does language popularity affect salary?
We saw which are the most popular programming languages, but what about salary? Are developers who use more popular languages better paid?
Let's try to figure that out by using the ConvertedComp (yearly salary in USD) and the Language column.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('data/2020.csv')

language_and_salary = df[['ConvertedComp', 'LanguageWorkedWith']]
language_and_salary = language_and_salary.dropna()
language_and_salary.head()

,ConvertedComp,LanguageWorkedWith
7,116000.0,Python;SQL
9,32315.0,HTML/CSS;Java;JavaScript;Python;SQL
10,40070.0,C#;JavaScript;Swift
11,14268.0,HTML/CSS;JavaScript
12,38916.0,C;JavaScript;Python


In [2]:
# First let's find the average salary each developer received by grouping the responses
mean_salaries = language_and_salary.groupby('LanguageWorkedWith').aggregate(['count','mean'])
# Set more meaningful column names
mean_salaries.columns = ['Users', 'Average salary']
mean_salaries.columns.name = 'Langauge'
mean_salaries.index.name = ''
mean_salaries = mean_salaries.sort_values('Average salary', ascending=False)
mean_salaries.head()

Langauge,Users,Average salary
,,
Bash/Shell/PowerShell;Java;JavaScript;Objective-C;Ruby;Swift,1,2000000.0
Bash/Shell/PowerShell;C;C#;Go;HTML/CSS;Java;JavaScript;Ruby,1,2000000.0
C#;C++;HTML/CSS;JavaScript;Perl;SQL;VBA,1,2000000.0
C#;Java;R;Scala;SQL,1,2000000.0
C;C++;Go;HTML/CSS;Java;JavaScript;Kotlin;Objective-C;PHP;Python;SQL;Swift;TypeScript,1,2000000.0


We grouped people by the language they used, but because the language in the survey is the whole stack of languages they used, we got information about some exceptional individuals. We really want to get information about every language. So let's split the survey results into different columns.

In [3]:
def split_column(column: pd.Series, separator: str = ';') -> pd.DataFrame:
    """
    INPUT:
    responses - pandas Series with multiple values per cell, separated by a delimiter
    separator - separator between the values in the cells

    OUTPUT:
    pandas DataFrame where for each unique value exists a column and its value is either 1 or 0
    """
    return column.str.get_dummies(sep=separator)

split_languages_salaries_df = split_column(language_and_salary['LanguageWorkedWith'])
split_languages_salaries_df['Salary'] = language_and_salary['ConvertedComp']
split_languages_salaries_df.head()

,Assembly,Bash/Shell/PowerShell,C,C#,C++,Dart,Go,HTML/CSS,Haskell,Java,...,Python,R,Ruby,Rust,SQL,Scala,Swift,TypeScript,VBA,Salary
7,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,116000.0
9,0,0,0,0,0,0,0,1,0,1,...,1,0,0,0,1,0,0,0,0,32315.0
10,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,40070.0
11,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,14268.0
12,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,38916.0


Now we have data we can work with. Let's try to determine the average salary for each language.

In [4]:
def aggregate_mean(df: pd.DataFrame, columns: [], agg_column: str) -> pd.DataFrame:
    """
    INPUT:
    df - pandas DataFrame with columns for which to find count and mean
    columns - list of column names for which to find count and mean
    agg_column - column for which to find mean

    OUTPUT:
    DataFrame with count and mean for each aggregated column
    """
    aggregated = {}
    for column in list(columns):
        mean = df[df[column] == 1].mean()[agg_column]
        count = len(df[df[column] == 1])
        aggregated[column] = (count, mean)
    return pd.DataFrame.from_dict(aggregated, orient='index')

salaries_df = aggregate_mean(split_languages_salaries_df, split_languages_salaries_df.columns[:-1], 'Salary')
salaries_df.columns = ['Users', 'Average salary']
salaries_df.columns.name = 'Language'
salaries_df.sort_values('Average salary', ascending=False)

Language,Users,Average salary
Julia,241,170101.531120
Perl,1072,155724.990672
Scala,1377,145749.003631
Ruby,2681,143067.032451
Rust,1628,138282.399877
Go,3322,134861.133955
Objective-C,1406,132170.721195
Bash/Shell/PowerShell,12514,123621.343455
R,1789,123614.177194
Assembly,1420,121457.115493


We can see some very interesting results here. The top 5 most popular programming languages we found in the other notebook were JavaScript, HTML/CSS, SQL, Python and Java. From those 4 (excluding Python) are all the way on the bottom of the salary pyramid, with Python being slightly above them.
This data doesn't give us exact information on what one's salary would be if they used only one language. What we get instead is an estimation of a developer's salary if they use a particular language alongside other languages, which we are not accounting for here. So the fact Julia programmers are very well paid does not mean that these programmers only use Julia as  all of them could also be using Python and the combination of these 2 languages could be bossting their salaries to top place. We will come back to this later and try to explore ways to better determine one's salary.

Before we move on, let's see how happy developers are using a particular language. We will do the same data processing as we did for salary.

In [5]:
language_and_satisfaction = df[['JobSat', 'LanguageWorkedWith']]
language_and_satisfaction = language_and_satisfaction.dropna()
# JobSat is a categorical field, so let's map it to numerical values
list(language_and_satisfaction['JobSat'].unique())

['Slightly satisfied',
 'Very dissatisfied',
 'Very satisfied',
 'Slightly dissatisfied',
 'Neither satisfied nor dissatisfied']

In [6]:
# Create a map with all possible options for Job Satisfaction.
# We will use grades from 0 to 4 to determine how happy developers are 
satisfaction_map = {
    'Very satisfied': 4,
    'Slightly satisfied': 3,
    'Neither satisfied nor dissatisfied': 2,
    'Slightly dissatisfied': 1,
    'Very dissatisfied': 0,
}

split_languages_satisfaction_df = split_column(language_and_satisfaction['LanguageWorkedWith'])
split_languages_satisfaction_df['Satisfaction'] = language_and_satisfaction['JobSat'].map(satisfaction_map).dropna()
split_languages_satisfaction_df.head()

,Assembly,Bash/Shell/PowerShell,C,C#,C++,Dart,Go,HTML/CSS,Haskell,Java,...,Python,R,Ruby,Rust,SQL,Scala,Swift,TypeScript,VBA,Satisfaction
0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,3
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,3
6,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,4
7,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1


In [7]:
# Use the function we wrote earlier to calculate the mean satisfaction and number of people using a given language 
satisfaction_df = aggregate_mean(split_languages_satisfaction_df, split_languages_satisfaction_df.columns[:-1], 'Satisfaction')
satisfaction_df.columns = ['Users', 'Satisfaction']
satisfaction_df.columns.name = 'Language'
# Our happiness scale is [0-4] so we scale to [0-100] to look at the percentages
satisfaction_df['Satisfaction'] = satisfaction_df['Satisfaction'] * 25
satisfaction_df.sort_values('Satisfaction', ascending=False)

Language,Users,Satisfaction
Julia,328,68.750000
R,2249,68.719431
Ruby,3240,68.441358
Bash/Shell/PowerShell,15146,68.092236
Haskell,728,68.028846
Rust,2027,67.254563
Assembly,1929,67.210990
Python,17569,67.209289
Scala,1699,67.157151
TypeScript,12232,67.129251


Seems like the Julia programmers who also earn the most are the happiest with their job. While JavaScript, HTML/CSS programmers are not that happy (3% less happy than the Julia programmers)